<a href="https://colab.research.google.com/github/herosunly/100-Days-Of-ML-Code/blob/master/CNN_metafeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = './drive/My Drive/data/'
import pickle
train_labels = pickle.load(open(path + 'train_label.pkl', 'rb'))
#train_seq = pickle.load(open(path + 'tr_api_type_seq.pkl', 'rb'))
#test_seq = pickle.load(open(path + 'te_api_type_seq.pkl', 'rb'))
train_seq = pickle.load(open(path + 'tr_seq.pkl', 'rb'))
test_seq = pickle.load(open(path + 'te_seq.pkl', 'rb'))

In [0]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Lambda, Embedding, Dropout, Activation, GRU, Bidirectional
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalAveragePooling1D, GlobalMaxPooling1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract, average
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

In [0]:
def TextCNN(max_len,max_cnt,embed_size,
            num_filters,kernel_size,
            conv_action,
            mask_zero):
    _input = Input(shape=(max_len,), dtype='int32')
    _embed = Embedding(max_cnt, embed_size, input_length=max_len, mask_zero=mask_zero)(_input)
    _embed = SpatialDropout1D(0.15)(_embed)
    warppers = []
    for _kernel_size in kernel_size:
        conv1d = Conv1D(filters=num_filters, kernel_size=_kernel_size, activation=conv_action)(_embed)
        warppers.append(GlobalMaxPooling1D()(conv1d))
                        
    fc = concatenate(warppers)
    fc = Dropout(0.5)(fc)
    #fc = BatchNormalization()(fc)
    fc = Dense(256, activation='relu')(fc)
    fc = Dropout(0.25)(fc)
    #fc = BatchNormalization()(fc) 
    preds = Dense(8, activation = 'softmax')(fc)
    
    model = Model(inputs=_input, outputs=preds)
    
    #model.compile(loss='categorical_crossentropy',
    #              optimizer='adam',
    #              metrics=['accuracy'])
    return model

In [0]:
def CNN(max_len, max_cnt, embed_size, mask_zero):
    _input = Input(shape=(max_len,), dtype='int32')
    _embed = Embedding(max_cnt, embed_size, input_length=max_len, mask_zero=mask_zero)(_input)
    _embed = SpatialDropout1D(0.4)(_embed)
    
    wrappers = []
    
    for ks in [3, 5, 7, 9, 11]:
        conv1d = Conv1D(filters=64, kernel_size=ks, activation='relu')(_embed)
        conv1d = Conv1D(filters=64, kernel_size=ks, activation='relu')(conv1d)    
        pool1d = MaxPooling1D(2)(conv1d)
        conv1d_2 = Conv1D(filters=128, kernel_size=ks, activation='relu')(pool1d)
        conv1d_2 = Conv1D(filters=128, kernel_size=ks, activation='relu')(conv1d_2)
        pool1d_2 = GlobalAveragePooling1D()(conv1d_2)
        wrappers.append(pool1d_2)
    
    fc = concatenate(wrappers)
    fc = Dropout(0.5)(fc)
    fc = Dense(256, activation='relu')(fc)
    fc = Dropout(0.5)(fc)
    #preds = Dense(8, activation='softmax')(fc)
    preds = Dense(2, activation='softmax')(fc)
        
    model = Model(inputs=_input, outputs=preds)
    
    return model
    

In [0]:
def CNN_LSTM(max_len, max_cnt, embed_size, mask_zero):
    _input = Input(shape=(max_len,), dtype='int32')
    _embed = Embedding(max_cnt, embed_size, input_length=max_len, mask_zero=mask_zero)(_input)
    _embed = SpatialDropout1D(0.15)(_embed)
    
    conv1d = Conv1D(filters=60, kernel_size=3, activation='relu')(_embed)
    pool1d = MaxPooling1D(2)(conv1d)
    conv1d_2 = Conv1D(filters=60, kernel_size=3, activation='relu')(pool1d)
    pool1d_2 = MaxPooling1D(2)(conv1d_2)
    lstm = LSTM(100, return_sequences=True)(pool1d_2)
    pool = GlobalAveragePooling1D()(lstm)
    
    fc = Dropout(0.5)(pool)
    preds = Dense(8, activation='softmax')(fc)
        
    model = Model(inputs=_input, outputs=preds)
    
    return model
    

In [0]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
max_len = 6000
max_cnt = 302
embed_size = 128
#embed_size = 17
num_filters = 64
kernel_size = [2,4,6,8,10,12,14]
conv_action = 'relu'
mask_zero = False
TRAIN = True

In [0]:
tf.logging.set_verbosity(0)
meta_train = np.zeros(shape = (len(train_seq),2))
meta_test = np.zeros(shape = (len(test_seq),2))
binary_labels = np.array([[1,0] if i == 0 else [0,1] for i in train_labels.argmax(1)])

for i,(tr_ind,te_ind) in enumerate(skf.split(train_seq, binary_labels.argmax(1))):
    print('FOLD: ', i + 1)
    print(len(te_ind),len(tr_ind))
    #model = TextCNN(max_len,max_cnt,embed_size,num_filters,kernel_size,conv_action,mask_zero)
    model = CNN(max_len, max_cnt, embed_size, mask_zero)
    #model = CNN_LSTM(max_len, max_cnt, 17, mask_zero)
    model_name = 'benchmark_cnn_fold_' + str(i)
    X_train, X_train_label = train_seq[tr_ind], binary_labels[tr_ind]
    X_val, X_val_label = train_seq[te_ind], binary_labels[te_ind]
    model_save_path = path + 'model_weight_final/%s_%s.hdf5'%(model_name, embed_size)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint(model_save_path, save_best_only=True, save_weights_only=True)

    strategy = tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR']))
    model = tf.contrib.tpu.keras_to_tpu_model(model, strategy)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    if TRAIN:
        model.fit(X_train, X_train_label,
                  validation_data=(X_val, X_val_label),
                  epochs=100, batch_size=64,
                  shuffle=True,
                  callbacks=[early_stopping]
                 )
    
    #model.load_weights(model_save_path)
    model.save_weights('/tmp/%s.h5' % model_name, overwrite=True)
    cpu_model = model.sync_to_cpu()
    pred_val = cpu_model.predict(X_val, batch_size=64, verbose=1)
    pred_test = cpu_model.predict(test_seq, batch_size=64, verbose=1)
    
    pred_train = cpu_model.predict(train_seq, batch_size=64, verbose=1)
    print(confusion_matrix(binary_labels.argmax(1), pred_train.argmax(1)))
        
    meta_train[te_ind] = pred_val
    meta_test += pred_test
    tf.keras.backend.clear_session()
meta_test /= 5.0


FOLD:  1
2778 11109
Train on 11109 samples, validate on 2778 samples
Epoch 1/100
11109/11109 [==============================] - 114s 10ms/sample - loss: 0.4857 - acc: 0.7666 - val_loss: 0.4159 - val_acc: 0.8192
Epoch 2/100
11109/11109 [==============================] - 10s 913us/sample - loss: 0.3597 - acc: 0.8517 - val_loss: 0.3999 - val_acc: 0.8318
Epoch 3/100
11109/11109 [==============================] - 10s 918us/sample - loss: 0.3259 - acc: 0.8646 - val_loss: 0.3180 - val_acc: 0.8822
Epoch 4/100
11109/11109 [==============================] - 10s 891us/sample - loss: 0.2877 - acc: 0.8736 - val_loss: 0.2766 - val_acc: 0.8811
Epoch 5/100
11109/11109 [==============================] - 10s 895us/sample - loss: 0.2384 - acc: 0.8919 - val_loss: 0.2241 - val_acc: 0.9218
Epoch 6/100
11109/11109 [==============================] - 10s 906us/sample - loss: 0.2031 - acc: 0.9139 - val_loss: 0.2019 - val_acc: 0.9413
Epoch 7/100
11109/11109 [==============================] - 10s 906us/sample - l

In [0]:
pd.to_pickle(meta_train, path + 'train_meta_cnn.pkl')
pd.to_pickle(meta_test, path + 'test_meta_cnn.pkl')